In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


2024-02-22 16:46:46.584612: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 16:46:47.230860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
def keras2TFlite(model_path,name):
    #load a pre-trained model
    with tf.device('/gpu:0'):
        model = tf.keras.models.load_model(model_path)
        #convert the model
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        #save the converted model
        with open('../../model/classification/tflite/'+name+'.tflite', 'wb') as f:
            f.write(tflite_model)
        print('Model converted successfully')
# keras2TFlite('../../model/classification/MobileNetV2_checkpoints.h5','MobileNetV2')
# keras2TFlite('../../model/classification/MobileNetV1_acc_checkpoints.h5','MobileNetV1')
# keras2TFlite('../../model/classification/DenseNet121_checkpoints.h5','DenseNet121')
# keras2TFlite('../../model/classification/EfficientNetV2B0_checkpoints.h5','EfficientNetV2B0')
# keras2TFlite('../../model/classification/NASNetMobile_checkpoints.h5','NASNetMobile')
# keras2TFlite('../../model/classification/ResNet50_checkpoints.h5','ResNet50')        

2024-02-22 16:51:43.504583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2024-02-22 16:51:45.723168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpsarrseo2/assets


INFO:tensorflow:Assets written to: /tmp/tmpsarrseo2/assets
2024-02-22 16:52:01.325612: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-22 16:52:01.325692: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-22 16:52:01.326001: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpsarrseo2
2024-02-22 16:52:01.346011: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-22 16:52:01.346038: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpsarrseo2
2024-02-22 16:52:01.433605: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-22 16:52:02.010203: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpsarrseo2
2024-02-22 16:52:02.169050: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

Model converted successfully


2024-02-22 16:52:26.667982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,1,1024]
	 [[{{node inputs}}]]
2024-02-22 16:52:26.677890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,1,1000]
	 [[{{node inputs}}]]
2024-02-22 16:52:27.233645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2024-

...

Model converted successfully


2024-02-22 16:57:49.169608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2024-02-22 16:57:51.789780: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpscnul_0b/assets


INFO:tensorflow:Assets written to: /tmp/tmpscnul_0b/assets
2024-02-22 16:58:11.926795: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-22 16:58:11.926878: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-22 16:58:11.927313: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpscnul_0b
2024-02-22 16:58:11.951958: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-22 16:58:11.952087: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpscnul_0b
2024-02-22 16:58:12.069409: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-22 16:58:12.940775: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpscnul_0b
2024-02-22 16:58:13.147809: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

Model converted successfully
